Importing the libraries

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the dataset

In [26]:
df=pd.read_csv("THREADS_REVIEWS.csv",nrows=10000)
df.head()

,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,068ccedd-f9b4-4b13-acd5-36320e14d29c,100023586722164617484,Je********to,"At this time it doesn't allow new accounts, on...",1,71,289.0.0.77.109,2023-07-05 22:53:12
1,7891117f-8c3e-4c1f-8480-293dc8fe4280,204908156566918819819,****,Incredible!,5,18,NaN,2023-07-05 22:53:18
2,427c4c1d-f242-4eac-ba10-5fb199372798,257377166405918921652,Ma***********fa,Not bad,3,15,NaN,2023-07-05 22:56:09
3,f4693442-f67e-47b1-b831-30c9aacecb77,193219204112065512952,***,First one to rate this app,5,2,NaN,2023-07-05 22:56:23
4,27208dca-4ec6-44bb-91ed-cbd9e5e893a6,306902324209474272757,Ni******es,this gotta be one of the apps of all time,5,23,289.0.0.77.109,2023-07-05 22:56:52


Cleaning the texts

In [27]:
df.drop(columns=["author_app_version","review_timestamp","review_id","pseudo_author_id","author_name","review_likes"],inplace=True)

In [28]:
df.head()

,review_text,review_rating
0,"At this time it doesn't allow new accounts, on...",1
1,Incredible!,5
2,Not bad,3
3,First one to rate this app,5
4,this gotta be one of the apps of all time,5


In [29]:
df['rated']=(df['review_rating']>2).astype(int)

In [30]:
df.head()

,review_text,review_rating,rated
0,"At this time it doesn't allow new accounts, on...",1,0
1,Incredible!,5,1
2,Not bad,3,1
3,First one to rate this app,5,1
4,this gotta be one of the apps of all time,5,1


In [31]:
df.drop(columns=["review_rating"],inplace=True)

In [32]:
df.head()

,review_text,rated
0,"At this time it doesn't allow new accounts, on...",0
1,Incredible!,1
2,Not bad,1
3,First one to rate this app,1
4,this gotta be one of the apps of all time,1


In [33]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus=[]
for i in range(0,10000):
  review=re.sub('[^a-zA-Z]',' ',df['review_text'][i])
  review=review.lower()
  review=review.split()
  ps=PorterStemmer()
  all_stopwords=stopwords.words('english')
  all_stopwords.remove('not')
  review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
  review=' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Creating the Bag of Words model

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=None)
x=cv.fit_transform(corpus).toarray()
y=df.iloc[:,1:].values

In [35]:
print(y)

[[0]
 [1]
 [1]
 ...
 [0]
 [1]
 [1]]


In [36]:
print(len(corpus))

10000


In [37]:
print(len(x[0]))

7572


In [38]:
print(y.shape)

(10000, 1)


In [39]:
print(x.shape)

(10000, 7572)


Splitting the dataset into the Training set and Test set

In [40]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

Training the Logistic Regression model on the Training set

In [41]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=0)

Predicting the Test set results

In [42]:
y_pred = classifier.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 1]
 [1 1]
 ...
 [1 1]
 [1 1]
 [1 1]]


Making the Confusion Matrix

In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 159  211]
 [  47 1583]]


0.871

In [44]:
print(x_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Prediciting Single Review As Positive or Negative

In [57]:
# Positive Review
#Input was a text and it is positive. Let's check what model predicts
new_review = 'Allow us to use multiple accounts at a time like "switch account" and also add a "like section" where we can see what we like or how many post we have liked.'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_x_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_x_test)
print(new_y_pred)

[1]


In [56]:
# Negative Review
#Input was a text and it is negative. Let's check what model predicts
new_review = 'Its just a copy of twitter with alot of restrictions'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_x_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_x_test)
print(new_y_pred)

[0]
